In [1]:
# This example uses data from scan_54 from cohere-ui/example_workspace
# This example does not use configuration files but defines parameters
# The experiment_dir is defined the same as in a cohere-ui framework for easy of use
import os
experiment_dir = os.path.join(os.path.dirname(os.getcwd()), 'example_workspace', 'scan_54')
print(experiment_dir)

/home/phoebus3/BFROSIK/coh-dev/cohere-ui/example_workspace/scan_54


In [2]:
# create detector class
import numpy as np
import os
import re
import glob
import cohere_core.utilities as ut
from abc import ABC, abstractmethod

class Detector(ABC):
    """
    Abstract class representing detector.
    """

    def __init__(self, name):
        self.name = name

    def dirs4scans(self, scans):
        # create empty results list that allocates a sub-list for each scan range
        scans_dirs_ranges = [[] for _ in range(len(scans))]
        sr_idx = 0
        scan_range = scans[sr_idx]
        scans_dirs = scans_dirs_ranges[sr_idx]

        # check for directories
        for scandir in sorted(os.listdir(self.data_dir)):
            scandir_full = ut.join(self.data_dir, scandir)
            if os.path.isdir(scandir_full):
                last_digits = re.search(r'\d+$', scandir)
                if last_digits is not None:
                    scan = int(last_digits.group())
                else:
                    continue
                if scan < scan_range[0]:
                    continue
                elif scan <= scan_range[-1]:
                    # scan within range
                    scans_dirs.append((scan, scandir_full))
                    if scan == scan_range[-1]:
                        sr_idx += 1
                        if sr_idx > len(scans) - 1:
                            break
                    scan_range = scans[sr_idx]
                    scans_dirs = scans_dirs_ranges[sr_idx]

        # remove empty sub-lists
        scans_dirs_ranges = [e for e in scans_dirs_ranges if len(e) > 0]
        return scans_dirs_ranges

    def get_scan_array(self, scan_info):
        """
        Reads/loads raw data file and applies correction.

        Reads raw data from a directory. The directory name is scan_info. The raw data is in form of 2D
        frames. The frames are read, corrected and stocked into 3D data

        :param scan_info: directory where the detector to retrieve data for a scan
        :return: corrected data array
        """
        slices_files = {}
        for file_name in os.listdir(scan_info):
            if file_name.endswith('tif'):
                fnbase = file_name[:-4]
            else:
                continue
            # for aps_34idc the file names end with the slice number, followed by 'tif' extension
            last_digits = re.search(r'\d+$', fnbase)
            if last_digits is not None:
                key = int(last_digits.group())
                slices_files[key] = ut.join(scan_info, file_name)

        ordered_keys = sorted(list(slices_files.keys()))
        ordered_slices = [self.correct_frame(slices_files[k]) for k in ordered_keys]

        return np.stack(ordered_slices, axis=-1)

class Detector_34idcTIM2(Detector):
    """
    Encapsulates "34idcTIM2" detector.
    """
    name = "34idcTIM2"
    dims = (512, 512)
    roi = (0, 512, 0, 512)
    pixel = (55.0e-6, 55e-6)
    pixelorientation = ('x+', 'y-')  # in xrayutilities notation
    whitefield = None
    darkfield = None
    raw_frame = None
    min_files = None  # defines minimum frame scans in scan directory
    Imult = None

    def __init__(self, **kwargs):
        super(Detector_34idcTIM2, self).__init__(self.name)
        # The detector attributes for background/whitefield/etc need to be set to read frames
        # this will capture things like data directory, whitefield_filename, etc.
        # keep parameters that are relevant to the detector
        self.data_dir = kwargs.get('data_dir')
        if 'roi' in kwargs:
            self.roi = kwargs.get('roi')
        if 'whitefield_filename' in kwargs:
            self.whitefield = ut.read_tif(kwargs.get('whitefield_filename'))
            # the code below is specific to TIM2 detector, excluding the correction of the weird pixels
            self.whitefield[255:257, 0:255] = 0  # wierd pixels on edge of seam (TL/TR). Kill in WF kills in returned frame as well.
            self.wfavg = np.average(self.whitefield)
            self.wfstd = np.std(self.whitefield)
            self.whitefield = np.where(self.whitefield < self.wfavg - 3 * self.wfstd, 0, self.whitefield)
            self.Imult = kwargs.get('Imult', self.wfavg)
            print('imult', self.Imult)
        if 'darkfield_filename' in kwargs:
            self.darkfield = ut.read_tif(kwargs.get('darkfield_filename'))
            if self.whitefield is not None:
                self.whitefield = np.where(self.darkfield > 1, 0, self.whitefield)  # kill known bad pixel

        self.min_files = kwargs.get('min_files', None)

    def correct_frame(self, filename):
        """
        Reads raw frame from a file, and applies correction for 34idcTIM2 detector, i.e. darkfield, whitefield,
        and seam.

        Parameters
        ----------
        filename : str
            data file name
        Returns
        -------
        frame : ndarray
            frame after correction
        """
        # roi is start,size,start,size
        # will be in imageJ coords, so might need to transpose,or just switch x-y
        # divide whitefield
        # blank out pixels identified in darkfield
        roislice1 = slice(self.roi[0], self.roi[0] + self.roi[1])
        roislice2 = slice(self.roi[2], self.roi[2] + self.roi[3])

        frame = ut.read_tif(filename)
        if self.whitefield is not None:
            frame = frame / self.whitefield[roislice1, roislice2] * self.Imult
        else:
            print('whitefield_filename not given, not correcting')
        if self.darkfield is not None:
            frame = np.where(self.darkfield[roislice1, roislice2] > 1, 0.0, frame)
        else:
            print('darkfield_filename not given, not correcting')

        frame = np.where(np.isfinite(frame), frame, 0)
        # insert 4 cols 5 rows if roi crosses asic boundary
        # frame, seam_added = self.insert_seam(frame)
        frame = np.where(np.isnan(frame), 0, frame)

        # if seam_added:
        #    frame = self.clear_seam(frame)
        return frame

def create_detector(det_name, **kwargs):
    if det_name == '34idcTIM2':
        return Detector_34idcTIM2(**kwargs)
    else:
        print(f'detector {det_name} not defined.')
        return None


dets = {'34idcTIM2' : Detector_34idcTIM2}

def get_pixel(det_name):
    return dets[det_name].pixel


def get_pixel_orientation(det_name):
    return dets[det_name].pixelorientation

In [3]:
# beamline preprocess

prep_params = {'data_dir' : "/home/phoebus3/BFROSIK/cohere/cohere-ui/example_data/AD34idcTIM2_example",
               'darkfield_filename' : "/home/phoebus3/BFROSIK/cohere/cohere-ui/example_data/dark.tif",
               'whitefield_filename' : "/home/phoebus3/BFROSIK/cohere/cohere-ui/example_data/whitefield.tif",
               'roi' : [0, 256, 0, 256, 1]}

det_obj = create_detector('34idcTIM2', **prep_params)
# get directory where scan data is
scan = 54
# The function datainfo4scans handles all cases, such as list of scan ranges.
#Therefore a single scan needs to be passed as a single sublist representing single scan range.
scans_datainfo = det_obj.dirs4scans([[54]])
print(scans_datainfo)
# For one scan the preprocessing is very simple and the code is copied here from beamlines.simple.preprocessor.
# For cases with multiple scans, scan ranges it would be best to import aps_34idc beamline preprocessor and call the process_batch function.
print(scans_datainfo[0])
arr = det_obj.get_scan_array(scans_datainfo[0][0][1])
# create the directory where the data should be saved and save it in tif file
save_dir = ut.join(experiment_dir, 'preprocessed_data')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_file = ut.join(save_dir, 'prep_data.tif')
ut.save_tif(arr, save_file)

imult 14688.119
[[(54, '/home/phoebus3/BFROSIK/cohere/cohere-ui/example_data/AD34idcTIM2_example/Staff20-1a_S0054')]]
[(54, '/home/phoebus3/BFROSIK/cohere/cohere-ui/example_data/AD34idcTIM2_example/Staff20-1a_S0054')]


/tmp/ipykernel_48730/2671937392.py:138: RuntimeWarning: divide by zero encountered in divide
  frame = frame / self.whitefield[roislice1, roislice2] * self.Imult
/tmp/ipykernel_48730/2671937392.py:138: RuntimeWarning: invalid value encountered in divide
  frame = frame / self.whitefield[roislice1, roislice2] * self.Imult


In [3]:
# standard preprocess
import cohere_core.data as fd
import os

# the conf was found in preprocessing segment
auto_data = False
binning = [1,1,1]
# The parameters cold be an empty dict if auto_data is True
data_params = {'intensity_threshold' : 2.0, 'binning' : binning}
# directory where the standard preprocessed data will be stored
data_dir = ut.join(experiment_dir, "phasing_data")
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
# the input data file that was beaamline preprocessed
data_file = ut.join(experiment_dir, "preprocessed_data", "prep_data.tif")
fd.prep(data_file, auto_data, **data_params)

data ready for reconstruction, data dims: (256, 256, 216)


{'intensity_threshold': 2.0, 'binning': [1, 1, 1]}

In [4]:
# phasing
import cohere_core.controller as rec

rec_params = {'reconstructions' : 1,
             'processing' : "auto", 
             'device' : [0],
             'algorithm_sequence' : "3*(20*ER+180*HIO)+20*ER",
             'hio_beta' : 0.9,
             'initial_support_area' : [0.5, 0.5, 0.5],
             'shrink_wrap_trigger' : [1, 1],
             'shrink_wrap_type' : "GAUSS",
             'shrink_wrap_threshold' : 0.1,
             'shrink_wrap_gauss_sigma' : 1.0,
             'twin_trigger' : [2],
             'twin_halves' : [0, 0],
             'progress_trigger' : [0, 20]}

# select the package to run reconstruction: 'cp' for cupy, 'np' for numpy, and 'torch' for torch.
pkg = 'cp'
# Select the GPU id to use
device = 0
worker = rec.create_rec(rec_params, data_file, pkg, device)
worker.iterate()
ph_res_dir = os.path.join(experiment_dir, 'results_phasing')
worker.save_res(ph_res_dir)


data shape (256, 256, 201)
------iter 0   error 1.0935584986718441e+25
------iter 20   error 0.050796107637310135
------iter 40   error 0.384782988852545
------iter 60   error 0.5390864562342266
------iter 80   error 0.6425999082591727
------iter 100   error 0.7108959979460208
------iter 120   error 0.7725437631017449
------iter 140   error 0.8249379087129503
------iter 160   error 0.8687145775957364
------iter 180   error 0.9130135124833625
------iter 200   error 0.9449882564863183
------iter 220   error 0.04034883615058646
------iter 240   error 0.399937706066848
------iter 260   error 0.5382219481098326
------iter 280   error 0.6635046932932045
------iter 300   error 0.7279876438737283
------iter 320   error 0.7863388575401514
------iter 340   error 0.8228744050382698
------iter 360   error 0.8789221924231558
------iter 380   error 0.9298628392146462
------iter 400   error 0.9631136766569821
------iter 420   error 0.039888611774000375
------iter 440   error 0.40582069762380696
-----

0

In [7]:
class Diffractometer_34idc():
    """
    Subclass of Diffractometer. Encapsulates "34idc" diffractometer.
    """
    name = "34idc"
    sampleaxes = ('y+', 'z-', 'y+')  # in xrayutilities notation
    detectoraxes = ('y+', 'x-')
    incidentaxis = (0, 0, 1)
    sampleaxes_name = ('Theta', 'Chi', 'Phi')
    sampleaxes_mne = ('th', 'chi', 'phi')
    detectoraxes_name = ('Delta', 'Gamma')
    detectoraxes_mne = ('delta', 'gamma')
    detectordist_name = 'camdist'
    detectordist_mne = 'detdist'

    def get_geometry(self, shape, scan, **kwargs):
        """
        Calculates geometry based on diffractometer and detector attributes and experiment parameters.

        Typically, the delta, gamma, theta, phi, chi, scanmot, scanmot_del, detdist, detector_name,
        energy values are retrieved from experiment spec file.
        They can be overridden by configuration.

        :param shape: tuple, shape of array
        :param scan: scan the geometry is calculated for
        :param kwargs: The **kwargs reflect configuration, and could contain delta, gamma, theta, phi, chi, scanmot,
            scanmot_del, detdist, detector_name, energy.
        :return: tuple
            (Trecip, Tdir)
        """
        import math as m
        import xrayutilities.experiment as xuexp
        
        params = kwargs
        print('params', params)

        binning = params.get('binning', [1, 1, 1])
        pixel = get_pixel(params['detector'])
        px = pixel[0] * binning[0]
        py = pixel[1] * binning[1]

        detdist = params['detdist'] / 1000.0  # convert to meters
        scanmot = params['scanmot'].strip()
        enfix = 1
        # if energy is given in kev convert to ev for xrayutilities
        energy = params['energy']
        if m.floor(m.log10(energy)) < 3:
            enfix = 1000
        energy = energy * enfix  # x-ray energy in eV

        if scanmot == 'en':
            scanen = np.array((energy, energy + params['scanmot_del'] * enfix))
        else:
            scanen = np.array((energy,))
        qc = xuexp.QConversion(self.sampleaxes, self.detectoraxes, self.incidentaxis, en=scanen)

        # compute for 4pixel (2x2) detector
        pixelorientation = get_pixel_orientation(params['detector'])
        qc.init_area(pixelorientation[0], pixelorientation[1], shape[0], shape[1], 2, 2,
                     distance=detdist, pwidth1=px, pwidth2=py)

        # I think q2 will always be (3,2,2,2) (vec, scanarr, px, py)
        # should put some try except around this in case something goes wrong.
        if scanmot == 'en':  # seems en scans always have to be treated differently since init is unique
            q2 = np.array(qc.area(params['th'], params['chi'], params['phi'], params['delta'], params['gamma'], deg=True))
        elif scanmot in self.sampleaxes_mne:  # based on scanmot args are made for qc.area
            args = []
            for sampleax in self.sampleaxes_mne:
                if scanmot == sampleax:
                    scanstart = params[scanmot]
                    args.append(np.array((scanstart, scanstart + params['scanmot_del'] * binning[2])))
                else:
                    args.append(params[sampleax])
            for axis in self.detectoraxes_mne:
                args.append(params[axis])

            q2 = np.array(qc.area(*args, deg=True))
        else:
            print("scanmot not in sample axes or energy, exiting")
            raise RuntimeError

        # I think q2 will always be (3,2,2,2) (vec, scanarr, px, py)
        Astar = q2[:, 0, 1, 0] - q2[:, 0, 0, 0]
        Bstar = q2[:, 0, 0, 1] - q2[:, 0, 0, 0]
        Cstar = q2[:, 1, 0, 0] - q2[:, 0, 0, 0]

        xtal = kwargs.get('xtal', False)
        if xtal:
            Trecip_cryst = np.zeros(9)
            Trecip_cryst.shape = (3, 3)
            Trecip_cryst[:, 0] = Astar * 10
            Trecip_cryst[:, 1] = Bstar * 10
            Trecip_cryst[:, 2] = Cstar * 10
            return Trecip_cryst, None

        # transform to lab coords from sample reference frame
        Astar = qc.transformSample2Lab(Astar, params['th'], params['chi'], params['phi']) * 10.0  # convert to inverse nm.
        Bstar = qc.transformSample2Lab(Bstar, params['th'], params['chi'], params['phi']) * 10.0
        Cstar = qc.transformSample2Lab(Cstar, params['th'], params['chi'], params['phi']) * 10.0

        denom = np.dot(Astar, np.cross(Bstar, Cstar))
        A = 2 * m.pi * np.cross(Bstar, Cstar) / denom
        B = 2 * m.pi * np.cross(Cstar, Astar) / denom
        C = 2 * m.pi * np.cross(Astar, Bstar) / denom

        Trecip = np.zeros(9)
        Trecip.shape = (3, 3)
        Trecip[:, 0] = Astar
        Trecip[:, 1] = Bstar
        Trecip[:, 2] = Cstar

        Tdir = np.zeros(9)
        Tdir.shape = (3, 3)
        Tdir = np.array((A, B, C)).transpose()

        return (Trecip, Tdir)



In [ ]:
# post processing
# this part will only create the vts file and will be extended in another notebook example
import os
import numpy as np

image = np.load(os.path.join(ph_res_dir, 'image.npy'))
support = np.load(os.path.join(ph_res_dir, 'support.npy'))

# The first step of visualization is creating vts file

# directory to save visualization results
save_dir = os.path.join(experiment_dir, "results_viz")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

instr_params = {'diffractometer' : "34idc",
                'energy' : 9.0,
                'delta' : 32.174,
                'gamma' : 12.6346,
                'detdist' : 500.0,
                'th' : 0.2200042,
                'chi' : 90.0,
                'phi' : -5.0,
                'scanmot' : "th",
                'scanmot_del' : 0.005,
                'detector' : "34idcTIM2"}

# add binning
viz_params = instr_params
viz_params.update({'binning' : binning})

# creeate diffractometer
diff_obj = Diffractometer_34idc()

# get geometry
geometry = diff_obj.get_geometry(image.shape, None, **instr_params)
(Trecip, Tdir) = geometry
print(Trecip, Tdir)
unwrap = False
crop = [.5,.5,.5]

from beamline_visualization import CXDViz
viz = CXDViz(crop, geometry)
viz.visualize(image, support, None, save_dir, unwrap)


In [ ]:
# add postprocessing